In [1]:
pip install vaderSentiment

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
from textblob import TextBlob
import re
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [5]:
MicrosoftR = pd.read_csv(r"C:\Users\47478\OneDrive\Dokumenter\Skule\Master\Microsoft\Microsoft_Ferdig.csv")

In [6]:
MicrosoftR.shape

(1812, 16)

In [7]:
MicrosoftR.head(3)

,Unnamed: 0,Date,Time,X.1,X,X_id,article_link,article_title,article_time,author_name,author_link,article_content,appears_in,symbols,related_articles,Time.1
0,2,2018-03-13,18:42:00,2,473,{'$oid': '5ab8d21e3efc2c03c520d00f'},https://www.nasdaq.com/article/is-oracle-orcl-...,Is Oracle (ORCL) Set to Beat Earnings Estimate...,{'$date': '2018-03-12T18:42:00Z'},"By Benjamin Rains, Zacks.com",https://www.nasdaq.com/author/zacks,Cloud computing has been one of the most well-...,"Investing,Earnings","AMZN,GOOGL,WDAY,MSFT,CRM",NaN,{'$date': '2018-03-12T18:42:00Z'}
1,4,2018-03-13,22:09:00,4,849,{'$oid': '5ab8d1dc3efc2c03c520cf8b'},https://www.nasdaq.com/article/1-smart-artific...,1 Smart Artificial Intelligence Bet You Can Ma...,{'$date': '2018-03-12T22:09:00Z'},"By Harsh Chauhan, Motley Fool",https://www.nasdaq.com/author/fool,"A report by IDC, which follows the tech indust...","PersonalFinance,Stocks","MSFT,AI,GOOGL,AMZN,GOOG",NaN,{'$date': '2018-03-12T22:09:00Z'}
2,6,2018-03-15,16:44:00,6,1313,{'$oid': '5ab8decb3efc2c03c520ec83'},https://www.nasdaq.com/article/3-large-cap-etf...,3 Large-Cap ETFs With Solid Inflows in March,{'$date': '2018-03-14T16:44:00Z'},"By Zacks Equity Research, Zacks.com",https://www.nasdaq.com/author/zacks,Wall Street has been recovering lately as fear...,"Investing,ETFs","AMZN,GOOGL,AAPL,MSFT,VV",NaN,{'$date': '2018-03-14T16:44:00Z'}


In [9]:
MicrosoftAksje = pd.read_csv(r"C:\Users\47478\OneDrive\Dokumenter\Skule\Master\Microsoft\Microsoft_Prices.csv")

In [10]:
MicrosoftAksje.shape

(8710, 8)

In [11]:
MicrosoftAksje.head(3)

,Date,Open,High,Low,Close,Adj Close,Volume,Label
0,1986-03-13,0.088542,0.101563,0.088542,0.097222,0.062055,1031788800,1
1,1986-03-14,0.097222,0.102431,0.097222,0.100694,0.064271,308160000,1
2,1986-03-17,0.100694,0.103299,0.100694,0.102431,0.065379,133171200,1


In [12]:
#Merge datasettene
merge = MicrosoftR.merge(MicrosoftAksje, how = "inner", on = "Date")

In [13]:
#Create a function to get the subjecticity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create a fuction to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

In [14]:
#Create two new columns "Subjectivity" and "Polarity"
merge["Subjectivity"] = merge["article_content"].apply(getSubjectivity)
merge["Polarity"] = merge["article_content"].apply(getPolarity)

In [16]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\47478\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [17]:
merge['article_content_without_stopwords'] = merge['article_content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [18]:
from nltk.stem import PorterStemmer

port = PorterStemmer()

In [20]:
def stem_words(article_content_without_stopwords):
    return " ".join([port.stem(word) for word in article_content_without_stopwords.split()])

merge["EtterStemming"] = merge["article_content_without_stopwords"].apply(lambda article_content_without_stopwords: stem_words(article_content_without_stopwords))
merge.head()

,Unnamed: 0,Date,Time,X.1,X,X_id,article_link,article_title,article_time,author_name,...,High,Low,Close,Adj Close,Volume,Label,Subjectivity,Polarity,article_content_without_stopwords,EtterStemming
0,2,2018-03-13,18:42:00,2,473,{'$oid': '5ab8d21e3efc2c03c520d00f'},https://www.nasdaq.com/article/is-oracle-orcl-...,Is Oracle (ORCL) Set to Beat Earnings Estimate...,{'$date': '2018-03-12T18:42:00Z'},"By Benjamin Rains, Zacks.com",...,97.239998,93.970001,94.410004,91.155731,35387800,0,0.475277,0.108063,Cloud computing one well-known drivers growth ...,cloud comput one well-known driver growth tech...
1,4,2018-03-13,22:09:00,4,849,{'$oid': '5ab8d1dc3efc2c03c520cf8b'},https://www.nasdaq.com/article/1-smart-artific...,1 Smart Artificial Intelligence Bet You Can Ma...,{'$date': '2018-03-12T22:09:00Z'},"By Harsh Chauhan, Motley Fool",...,97.239998,93.970001,94.410004,91.155731,35387800,0,0.440061,0.169724,"A report IDC, follows tech industry, estimates...","a report idc, follow tech industry, estim worl..."
2,10,2018-03-13,16:12:00,10,1660,{'$oid': '5ab8d2b83efc2c03c520d159'},https://www.nasdaq.com/article/3-strong-buy-se...,3 Strong Buy Semiconductor Stocks to Consider Now,{'$date': '2018-03-12T16:12:00Z'},"By Ryan McQueeney, Zacks.com",...,97.239998,93.970001,94.410004,91.155731,35387800,0,0.489353,0.146362,Semiconductor stocks battered recent market se...,semiconductor stock batter recent market sell-...
3,8,2018-03-13,07:05:00,8,1472,{'$oid': '5ab8d0693efc2c03c520cc37'},https://www.nasdaq.com/article/slack-needs-to-...,Slack Needs to Worry About Microsoft Teams,{'$date': '2018-03-13T07:05:00Z'},"By Timothy Green, Motley Fool",...,97.239998,93.970001,94.410004,91.155731,35387800,0,0.508590,0.175829,"Slack, software company focused workplace mess...","slack, softwar compani focus workplac messagin..."
4,6,2018-03-15,16:44:00,6,1313,{'$oid': '5ab8decb3efc2c03c520ec83'},https://www.nasdaq.com/article/3-large-cap-etf...,3 Large-Cap ETFs With Solid Inflows in March,{'$date': '2018-03-14T16:44:00Z'},"By Zacks Equity Research, Zacks.com",...,94.580002,92.830002,94.180000,90.933655,27611000,1,0.532905,0.115972,Wall Street recovering lately fears aggressive...,wall street recov late fear aggress rate hike ...


In [21]:
#Create a function to get the sentiment scores
def getSIA(EtterStemming):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(EtterStemming)
    return sentiment

In [22]:
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\47478\AppData\Roaming\nltk_data...


True

In [23]:
#Get the sentiment scores for each day
compound = []
neg = []
pos = []
neu = []
SIA = 0

for i in range(0, len(merge["EtterStemming"])):
    SIA = getSIA(merge["EtterStemming"][i])
    compound.append(SIA["compound"])
    neg.append(SIA["neg"])
    neu.append(SIA["neu"])
    pos.append(SIA["pos"])

In [24]:
#Store the sentiment scores in the merge data set:
merge["Compound"] = compound
merge["Negative"] = neg
merge["Neutral"] = neu
merge["Positive"] = pos

In [25]:
#Show the merge data
merge.head(3)

,Unnamed: 0,Date,Time,X.1,X,X_id,article_link,article_title,article_time,author_name,...,Volume,Label,Subjectivity,Polarity,article_content_without_stopwords,EtterStemming,Compound,Negative,Neutral,Positive
0,2,2018-03-13,18:42:00,2,473,{'$oid': '5ab8d21e3efc2c03c520d00f'},https://www.nasdaq.com/article/is-oracle-orcl-...,Is Oracle (ORCL) Set to Beat Earnings Estimate...,{'$date': '2018-03-12T18:42:00Z'},"By Benjamin Rains, Zacks.com",...,35387800,0,0.475277,0.108063,Cloud computing one well-known drivers growth ...,cloud comput one well-known driver growth tech...,0.9633,0.000,0.901,0.099
1,4,2018-03-13,22:09:00,4,849,{'$oid': '5ab8d1dc3efc2c03c520cf8b'},https://www.nasdaq.com/article/1-smart-artific...,1 Smart Artificial Intelligence Bet You Can Ma...,{'$date': '2018-03-12T22:09:00Z'},"By Harsh Chauhan, Motley Fool",...,35387800,0,0.440061,0.169724,"A report IDC, follows tech industry, estimates...","a report idc, follow tech industry, estim worl...",0.9892,0.048,0.823,0.130
2,10,2018-03-13,16:12:00,10,1660,{'$oid': '5ab8d2b83efc2c03c520d159'},https://www.nasdaq.com/article/3-strong-buy-se...,3 Strong Buy Semiconductor Stocks to Consider Now,{'$date': '2018-03-12T16:12:00Z'},"By Ryan McQueeney, Zacks.com",...,35387800,0,0.489353,0.146362,Semiconductor stocks battered recent market se...,semiconductor stock batter recent market sell-...,0.9948,0.016,0.844,0.140


In [28]:
merge.to_csv("MicrosoftDato")

In [26]:
#Create a list of column to keep
keep_columns = ["Open", "Low", "High", "Volume", "Subjectivity", "Polarity", "Compound", "Negative", "Neutral", "Positive", "Label"]
df = merge[keep_columns]
df

,Open,Low,High,Volume,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label
0,97.000000,93.970001,97.239998,35387800,0.475277,0.108063,0.9633,0.000,0.901,0.099,0
1,97.000000,93.970001,97.239998,35387800,0.440061,0.169724,0.9892,0.048,0.823,0.130,0
2,97.000000,93.970001,97.239998,35387800,0.489353,0.146362,0.9948,0.016,0.844,0.140,0
3,97.000000,93.970001,97.239998,35387800,0.508590,0.175829,0.9906,0.062,0.803,0.135,0
4,93.529999,92.830002,94.580002,27611000,0.532905,0.115972,0.9877,0.074,0.781,0.145,1
...,...,...,...,...,...,...,...,...,...,...,...
1423,31.620001,31.250000,31.650000,63213000,0.461096,0.078854,0.9698,0.041,0.830,0.129,0
1424,32.869999,32.150002,32.869999,40685000,0.373844,-0.029014,0.7269,0.029,0.924,0.047,0
1425,37.240002,37.060001,38.220001,88948800,0.343695,0.045125,0.9300,0.040,0.867,0.094,1
1426,62.500000,62.240002,63.419998,35718900,0.400040,0.049240,0.7184,0.061,0.840,0.099,1


In [29]:
MicrosoftTest = pd.read_csv(r"C:\Users\47478\OneDrive\Dokumenter\Skule\Master\Microsoft\TestMicrosoft.csv")

In [30]:
MicrosoftTest.shape

(286, 32)

In [31]:
MicrosoftTest.head(3)

,NA,Unnamed: 0,Date,Time,X.1,X,X_id,article_link,article_title,article_time,...,Volume,Label,Subjectivity,Polarity,article_content_without_stopwords,EtterStemming,Compound,Negative,Neutral,Positive
0,758,2104,2016-07-20,16:50:56,2104,136361,{'$oid': '5aca24533efc2c02e719de38'},https://www.nasdaq.com/article/why-johnson-joh...,"Why Johnson & Johnson (JNJ), McDonald’s Corpor...",{'$date': '2016-07-19T16:50:56Z'},...,89893300,0,0.447931,0.167139,InvestorPlace InvestorPlace - Stock Market New...,"investorplac investorplac - stock market news,...",0.9844,0.018,0.877,0.105
1,759,2945,2016-07-20,18:46:00,2945,187575,{'$oid': '5acf903c3efc2c18c2b6bc81'},https://www.nasdaq.com/article/trimble-chinas-...,Trimble & China's CSCEC Tie Up for Mixed Reali...,{'$date': '2016-07-19T18:46:00Z'},...,89893300,0,0.398984,0.068981,Trimble Navigation Limited TRMB CSCEC No. 1 Bu...,trimbl navig limit trmb cscec no. 1 build grou...,0.9664,0.010,0.883,0.107
2,760,3221,2016-07-20,17:23:00,3221,206011,{'$oid': '5acfd8183efc2c18c2b8759e'},https://www.nasdaq.com/article/technology-stoc...,Technology Stocks to Watch This Earnings Seaso...,{'$date': '2016-07-19T17:23:00Z'},...,89893300,0,0.555122,0.078125,Is time tech?We take look three important tech...,is time tech?w take look three import tech com...,0.9477,0.032,0.851,0.117


In [32]:
MicrosoftTrain = pd.read_csv(r"C:\Users\47478\OneDrive\Dokumenter\Skule\Master\Microsoft\TrainMicrosoft.csv")

In [33]:
MicrosoftTrain.shape

(1142, 32)

In [34]:
MicrosoftTrain.head(3)

,NA,Unnamed: 0,Date,Time,X.1,X,X_id,article_link,article_title,article_time,...,Volume,Label,Subjectivity,Polarity,article_content_without_stopwords,EtterStemming,Compound,Negative,Neutral,Positive
0,1138,3954,2012-01-12,17:00:01,3954,251169,{'$oid': '5ad0815f3efc2c18c2bc7f4d'},https://www.nasdaq.com/article/ea-set-to-relea...,EA Set to Release New Golf Game - Analyst Blog,{'$date': '2012-01-11T17:00:01Z'},...,49370800,1,0.474964,0.112476,Video game developer publisher Electronic Arts...,video game develop publish electron art inc. (...,0.9902,0.026,0.830,0.144
1,1112,3847,2012-01-13,17:02:57,3847,245258,{'$oid': '5ad073813efc2c18c2bc2dd3'},https://www.nasdaq.com/article/3-nasdaq-stocks...,3 Nasdaq Stocks for Short-Term Traders,{'$date': '2012-01-12T17:02:57Z'},...,60196100,1,0.436944,0.104343,While Microsoft ( MSFT ) Sony ( SNE ) making p...,while microsoft ( msft ) soni ( sne ) make pre...,0.7959,0.070,0.823,0.107
2,1113,5756,2012-01-13,17:23:01,5756,368115,{'$oid': '5ad2a9ec3efc2c25783758b6'},https://www.nasdaq.com/article/eas-strategic-g...,EA's Strategic Game Promotion - Analyst Blog,{'$date': '2012-01-12T17:23:01Z'},...,60196100,1,0.412778,0.042500,Electronic Arts Inc. ( EA ) devised unique pla...,electron art inc. ( ea ) devis uniqu plan prom...,0.9670,0.024,0.862,0.114


In [40]:
#Create a list of column to keep
keep_columns1 = ["Open", "Low", "High", "Volume", "Subjectivity", "Polarity", "Compound", "Negative", "Neutral", "Positive", "Label"]
df1 = MicrosoftTest[keep_columns1]
df1

,Open,Low,High,Volume,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label
0,56.150002,55.529999,56.840000,89893300,0.447931,0.167139,0.9844,0.018,0.877,0.105,0
1,56.150002,55.529999,56.840000,89893300,0.398984,0.068981,0.9664,0.010,0.883,0.107,0
2,56.150002,55.529999,56.840000,89893300,0.555122,0.078125,0.9477,0.032,0.851,0.117,0
3,56.150002,55.529999,56.840000,89893300,0.454654,0.126862,0.9897,0.023,0.837,0.140,0
4,56.150002,55.529999,56.840000,89893300,0.345879,0.074283,0.9872,0.000,0.804,0.196,0
...,...,...,...,...,...,...,...,...,...,...,...
281,91.269997,89.660004,91.750000,38604700,0.411883,0.068289,0.9261,0.058,0.856,0.085,0
282,91.269997,89.660004,91.750000,38604700,0.484534,0.041175,0.9136,0.008,0.914,0.078,0
283,89.500000,87.080002,90.459999,44068900,0.523148,-0.048148,0.2960,0.000,0.937,0.062,0
284,90.610001,90.400002,94.000000,56396800,0.510927,0.034368,0.2468,0.031,0.926,0.043,1


In [41]:
#Create the feature data set
X = df1
X = np.array(X.drop(["Label"], 1))

#Create the target data set
y = np.array(df1["Label"])

C:\Users\47478\AppData\Local\Temp/ipykernel_11548/3997290794.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X = np.array(X.drop(["Label"], 1))


In [42]:
X

array([[5.6150002e+01, 5.5529999e+01, 5.6840000e+01, ..., 1.8000000e-02,
        8.7700000e-01, 1.0500000e-01],
       [5.6150002e+01, 5.5529999e+01, 5.6840000e+01, ..., 1.0000000e-02,
        8.8300000e-01, 1.0700000e-01],
       [5.6150002e+01, 5.5529999e+01, 5.6840000e+01, ..., 3.2000000e-02,
        8.5100000e-01, 1.1700000e-01],
       ...,
       [8.9500000e+01, 8.7080002e+01, 9.0459999e+01, ..., 0.0000000e+00,
        9.3700000e-01, 6.2000000e-02],
       [9.0610001e+01, 9.0400002e+01, 9.4000000e+01, ..., 3.1000000e-02,
        9.2600000e-01, 4.3000000e-02],
       [9.0610001e+01, 9.0400002e+01, 9.4000000e+01, ..., 4.0000000e-03,
        8.2600000e-01, 1.7000000e-01]])

In [43]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1],
      dtype=int64)

In [44]:
x_test = X

In [45]:
len(x_test)

286

In [46]:
y_test = y

In [48]:
len(y_test)

286

In [49]:
#Create a list of column to keep
keep_columns2 = ["Open", "Low", "High", "Volume", "Subjectivity", "Polarity", "Compound", "Negative", "Neutral", "Positive", "Label"]
df2 = MicrosoftTrain[keep_columns2]
df2

,Open,Low,High,Volume,Subjectivity,Polarity,Compound,Negative,Neutral,Positive,Label
0,27.870001,27.650000,28.020000,49370800,0.474964,0.112476,0.9902,0.026,0.830,0.144,1
1,27.930000,27.790001,28.250000,60196100,0.436944,0.104343,0.7959,0.070,0.823,0.107,1
2,27.930000,27.790001,28.250000,60196100,0.412778,0.042500,0.9670,0.024,0.862,0.114,1
3,28.820000,28.750000,29.740000,165902900,0.360816,0.081318,0.9852,0.030,0.829,0.142,1
4,28.820000,28.750000,29.740000,165902900,0.348056,0.059833,0.7840,0.061,0.847,0.092,1
...,...,...,...,...,...,...,...,...,...,...,...
1137,53.950001,53.209999,54.000000,32024400,0.519841,0.030159,0.2960,0.000,0.937,0.062,0
1138,53.700001,53.549999,54.340000,31433900,0.528232,0.062512,0.9870,0.013,0.894,0.093,1
1139,53.700001,53.549999,54.340000,31433900,0.213019,0.013586,0.9982,0.031,0.872,0.097,1
1140,53.709999,52.930000,53.900002,53336500,0.472899,0.149160,0.9201,0.117,0.737,0.146,0


In [50]:
#Create the feature data set
X1 = df2
X1 = np.array(X1.drop(["Label"], 1))

#Create the target data set
y1 = np.array(df2["Label"])

C:\Users\47478\AppData\Local\Temp/ipykernel_11548/791745654.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X1 = np.array(X1.drop(["Label"], 1))


In [51]:
X1

array([[2.7870001e+01, 2.7650000e+01, 2.8020000e+01, ..., 2.6000000e-02,
        8.3000000e-01, 1.4400000e-01],
       [2.7930000e+01, 2.7790001e+01, 2.8250000e+01, ..., 7.0000000e-02,
        8.2300000e-01, 1.0700000e-01],
       [2.7930000e+01, 2.7790001e+01, 2.8250000e+01, ..., 2.4000000e-02,
        8.6200000e-01, 1.1400000e-01],
       ...,
       [5.3700001e+01, 5.3549999e+01, 5.4340000e+01, ..., 3.1000000e-02,
        8.7200000e-01, 9.7000000e-02],
       [5.3709999e+01, 5.2930000e+01, 5.3900002e+01, ..., 1.1700000e-01,
        7.3700000e-01, 1.4600000e-01],
       [5.3709999e+01, 5.2930000e+01, 5.3900002e+01, ..., 2.2000000e-02,
        8.8200000e-01, 9.6000000e-02]])

In [52]:
y1

array([1, 1, 1, ..., 1, 0, 0], dtype=int64)

In [53]:
x_train = X1

In [54]:
len(X1)

1142

In [56]:
y_train = y1

In [70]:
len(y1)

1142

In [58]:
#Create and train the model
model = LinearDiscriminantAnalysis().fit(x_train, y_train)

In [59]:
#Show the models predictions
predictions = model.predict(x_test)
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1],
      dtype=int64)

In [60]:
y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1],
      dtype=int64)

In [61]:
#Show the model metrics
print( classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.74      0.86      0.80       150
           1       0.81      0.67      0.73       136

    accuracy                           0.77       286
   macro avg       0.78      0.76      0.77       286
weighted avg       0.78      0.77      0.77       286



In [62]:
#Create and train the model (Naive Bayes)
from sklearn.naive_bayes import GaussianNB
modelNB = GaussianNB()

In [63]:
#Train the model
modelNB.fit(x_train, y_train)

GaussianNB()

In [64]:
#Accuracy for modellen
modelNB.score(x_test, y_test)

0.5174825174825175

In [65]:
x_test[:10]

array([[5.61500020e+01, 5.55299990e+01, 5.68400000e+01, 8.98933000e+07,
        4.47930658e-01, 1.67138957e-01, 9.84400000e-01, 1.80000000e-02,
        8.77000000e-01, 1.05000000e-01],
       [5.61500020e+01, 5.55299990e+01, 5.68400000e+01, 8.98933000e+07,
        3.98984349e-01, 6.89810190e-02, 9.66400000e-01, 1.00000000e-02,
        8.83000000e-01, 1.07000000e-01],
       [5.61500020e+01, 5.55299990e+01, 5.68400000e+01, 8.98933000e+07,
        5.55121528e-01, 7.81250000e-02, 9.47700000e-01, 3.20000000e-02,
        8.51000000e-01, 1.17000000e-01],
       [5.61500020e+01, 5.55299990e+01, 5.68400000e+01, 8.98933000e+07,
        4.54653680e-01, 1.26862374e-01, 9.89700000e-01, 2.30000000e-02,
        8.37000000e-01, 1.40000000e-01],
       [5.61500020e+01, 5.55299990e+01, 5.68400000e+01, 8.98933000e+07,
        3.45879247e-01, 7.42825987e-02, 9.87200000e-01, 0.00000000e+00,
        8.04000000e-01, 1.96000000e-01],
       [5.61500020e+01, 5.55299990e+01, 5.68400000e+01, 8.98933000e+07,
   

In [66]:
#Faktisk oppgang eller nedgang
y_test[:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0], dtype=int64)

In [67]:
#Sjekker om vi modellen predikerer rett
modelNB.predict(x_test[:100])

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], dtype=int64)

In [68]:
#Sjekker sannsynligheten for oppgang og nedgang
Bayes = modelNB.predict_proba(x_test)

In [69]:
Bayes

array([[0.62064864, 0.37935136],
       [0.62064864, 0.37935136],
       [0.62064864, 0.37935136],
       [0.62064864, 0.37935136],
       [0.62064864, 0.37935136],
       [0.62064864, 0.37935136],
       [0.62064863, 0.37935137],
       [0.47706233, 0.52293767],
       [0.47706233, 0.52293767],
       [0.47706233, 0.52293767],
       [0.47706233, 0.52293767],
       [0.47706233, 0.52293767],
       [0.47706233, 0.52293767],
       [0.47706233, 0.52293767],
       [0.47706233, 0.52293767],
       [0.4763927 , 0.5236073 ],
       [0.4763927 , 0.5236073 ],
       [0.47639269, 0.52360731],
       [0.47247458, 0.52752542],
       [0.47657461, 0.52342539],
       [0.47657461, 0.52342539],
       [0.4765746 , 0.5234254 ],
       [0.47657461, 0.52342539],
       [0.48288023, 0.51711977],
       [0.48288023, 0.51711977],
       [0.48288023, 0.51711977],
       [0.48288023, 0.51711977],
       [0.48288023, 0.51711977],
       [0.4747557 , 0.5252443 ],
       [0.4747557 , 0.5252443 ],
       [0.